1. 是否要单独选择Restaurants这个category，数据量少很多，从七万条到只有三百多条；
2. 但如果看所有category，有category的business id也才五百多条；
3. 到底要不要看category？看哪个category？我的想法是看restaurants。
哦哦没事了，因为是sample data


In [1]:
import sparknlp
import pandas as pd
sparknlp.start()

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
#create Spark session
spark = SparkSession.builder.appName('YelpML').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_listenablefuture-9999.0-empty-to-avoid-conflict-

In [ ]:
# Import Data
dataDir = "gs://msca-bdp-student-gcs/group2/yelp-datasample2"
business = spark.read.json(dataDir + "/sample_business")
review = spark.read.json(dataDir + "/sample_review")

## Overview

### Join review df and business df to find the category information for all kinds of businesses

In [ ]:
review.show(5)

In [ ]:
review.count()

In [ ]:
review = review.select('business_id', 'text', 'stars')

In [ ]:
review.show(5)

In [ ]:
business.show(5)

In [ ]:
business = business.select('business_id', 'categories')

In [ ]:
business.show(5)

In [ ]:
from pyspark.sql.functions import explode, split
business = business.withColumn('categories_split', explode(split('categories', ', ')))

In [ ]:
business.show()

### Choose the reviews of restaurant category for furthur analysis

In [ ]:
from pyspark.sql.functions import col
business = business.where(col('categories_split') == 'Restaurants').select('business_id', 'categories_split')

In [ ]:
df = review.join(business, on = 'business_id', how = 'inner')

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
df = df.repartition(16)

### Review EDA

In [ ]:
df1 = df.groupBy('stars').count().orderBy('stars', ascending = [1])

In [ ]:
df1.show()

In [ ]:
pdf = df1.toPandas()

In [ ]:
pdf.plot(kind = 'bar', x = 'stars', y = 'count', rot = 360)

We have observed a trend where 5-star reviews receive the highest level of popularity. Additionally, we've noticed that 1-star reviews are more frequent compared to reviews with 2 or 3 stars. It can be assumed that customers are more inclined to go through the effort of writing a review when they are either extremely satisfied or extremely dissatisfied.

## Data Preprocessing

In [ ]:
language_detector = LanguageDetectorDL.pretrained("ld_tatoeba_bigru_21", "xx")\
.setInputCols(["text"])\
.setOutputCol("language")
languagePipeline = Pipeline(stages=[language_detector])


In [ ]:
df2 = languagePipeline.fit(df).transform

In [ ]:
from sparknlp.annotator import LanguageDetectorDL
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline

In [ ]:
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

# Load the pre-trained language detection model
language_detector = LanguageDetectorDL.pretrained("ld_wiki_20", "xx") \
    .setInputCols(["document"]) \
    .setOutputCol("language")

# Build the pipeline
languagePipeline = Pipeline(stages=[documentAssembler, language_detector])

# Fit the pipeline to your DataFrame
df2 = languagePipeline.fit(df).transform(df)

# # Create a LightPipeline using SparkContext and DataFrame
# spark_context = SparkContext.getOrCreate()
# light_pipeline = SparkContext.toLightPipeline(model, df)

# # Use the light pipeline to annotate your text
# result = light_pipeline.fullAnnotate("Spark NLP est une bibliothèque de traitement de texte...")


In [ ]:
# Initialize Spark NLP components
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
sentenceDetector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

# Load the pre-trained language detection model
language_detector = LanguageDetectorDL.pretrained("ld_tatoeba_bigru_21", "xx") \
    .setInputCols(["sentence"]) \
    .setOutputCol("language")

# Build the pipeline
languagePipeline = Pipeline(stages=[documentAssembler, sentenceDetector, language_detector])

# Fit the pipeline and create a LightPipeline
languageLightPipeline = LightPipeline(languagePipeline.fit(df))


In [ ]:
#https://sparknlp.org/2020/12/05/detect_language_21_xx.html
from sparknlp.pretrained import PretrainedPipeline
pipeline = PretrainedPipeline("detect_language_21", lang = "xx")

In [ ]:
# Define a UDF using the F.udf function
def detect_language_udf(text):
    result = pipeline.annotate(text)
    return result['language'][0] if result and 'language' in result else None

detect_language_spark_udf = F.udf(detect_language_udf, StringType())

# Apply the UDF to the 'text' column and create a new column 'detected_language'
df_with_language = df.withColumn("detected_language", detect_language_spark_udf(df["text"]))

# Show the resulting DataFrame with detected languages
df_with_language.show(truncate=False)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def detect_language(text):
    result = pipeline.annotate(text)
    return result['language'][1]

detect_language_udf = udf(detect_language, StringType())

In [ ]:
def detect_language_udf(text_series: pd.Series) -> pd.Series:
    result = []
    for text in text_series:
        annotations = pipeline.annotate(text)
        result.append(annotations['language'][0])
    return pd.Series(result)

In [ ]:
df = df.withColumn('language', detect_language_udf(df['text']))

In [ ]:
review.count()

In [ ]:
review = review.repartition(16)

In [ ]:
review.rdd.getNumPartitions()

In [ ]:
review_sample = review.sample(fraction=0.02, seed=42)

In [ ]:
review_sample.count()

In [ ]:
review_sample.show()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Word2Vec
from pyspark.ml.regression import LinearRegression
# from sparknlp.base import DocumentAssembler, Finisher
# from sparknlp.annotator import Tokenizer, Normalizer

In [ ]:
review_sample = review_sample.select('text', 'stars')

## Preprocessing

Includes: tokenizer, normalizer, and lemmatizer

In [ ]:
# Initialize SparkNLP components
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
normalizer = Normalizer().setInputCols(["token"]).setOutputCol("normalized").setLowercase(True) \
    .setCleanupPatterns(["""[^\w\d\s]"""])
lemmatizer = LemmatizerModel.pretrained().setInputCols(["normalized"]).setOutputCol("lemmatized")
finisher = Finisher().setInputCols(['lemmatized']).setOutputCols(["finished"])

# Create a SparkNLP pipeline
pipeline = Pipeline(stages=[document_assembler, tokenizer, normalizer, lemmatizer, finisher])



# Fit and transform the data
processed_data = pipeline.fit(review_sample).transform(review_sample)

# processed_data = processed_data.withColumn("normalized_tokens", F.expr("transform(normalized, x -> x.result)"))

In [ ]:
processed_data = processed_data.drop('text')

In [ ]:
processed_data.show()

In [ ]:
# Use Word2Vec to convert text into dense vectors
word2vec = Word2Vec(vectorSize=100, inputCol="finished", outputCol="features")
word2vec_model = word2vec.fit(processed_data)
word2vec_data = word2vec_model.transform(processed_data)

# Split the data into training and test sets
(training_data, test_data) = word2vec_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Initialize and train the linear regression model
lr = LinearRegression(featuresCol="features", labelCol="stars")
lr_model = lr.fit(training_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Evaluate the model (optional)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="stars", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
user.count()